In [11]:
#Importing packages and checking GPU
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split

if torch.cuda.is_available():
	print("PyTorch is using the GPU")
	GPUCount = torch.cuda.device_count()
	print(f"Found {GPUCount} GPUs")

	for i in range(GPUCount):
		print(f"GPU {i} found: {torch.cuda.get_device_name(i)}")

	device = torch.device("cuda:0")
else:
	print("PyTorch is using the CPU")
	device = torch.device("cpu")

print(f"Selected Device: {device}")

PyTorch is using the GPU
Found 1 GPUs
GPU 0 found: NVIDIA GeForce RTX 5070 Laptop GPU
Selected Device: cuda:0


In [12]:
#Load BuiltIn dataset
Transform = transforms.Compose(
	[transforms.ToTensor(),
  transforms.Normalize((0.5,),(0.5,))])
Transform = transforms.Compose(
	[transforms.ToTensor(),
  transforms.Normalize((0.5,),(0.5,))])

TrainSet = torchvision.datasets.EMNIST(root='./data',train=True,download=True,transform=Transform,split="letters",target_transform=lambda y:y-1)
TrainLoader = DataLoader(TrainSet, batch_size=128, shuffle=True)

TestSet = torchvision.datasets.EMNIST(root='./data',train=False,download=True,transform=Transform,split="letters",target_transform=lambda y:y-1)
TestLoader = DataLoader(TestSet, batch_size=128, shuffle=False)

In [13]:
#Creating Model

#Define architecture
class EMNISTModel(nn.Module):
	def __init__(self):
		super(EMNISTModel,self).__init__()

		#Layer1
		self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3)#Size does not change
		self.relu1 = nn.ReLU()
		self.pool1 = nn.MaxPool2d(kernel_size=2)#Size halves into 14x14

		#Layer2
		self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
		self.relu2 = nn.ReLU()
		self.pool2 = nn.MaxPool2d(kernel_size=2)#Size halves into 7x7

		#Layer3
		self.conv3 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3)

		#Layer4
		self.flatten = nn.Flatten()

		#Layer5
		self.fc1 = nn.Linear(in_features=64*3*3,out_features=64)
		self.relu3 = nn.ReLU()

		#Layer6
		self.fc2 = nn.Linear(in_features=64,out_features=26)

	def forward(self,x):

		#Pass through Layer1
		x = self.pool1(self.relu1(self.conv1(x)))

		#Pass through Layer2
		x = self.pool2(self.relu2(self.conv2(x)))

		#Pass through Layer3
		x = self.conv3(x)

		#Pass through Layer4
		x = self.flatten(x)

		#Pass through Layer5
		x = self.relu3(self.fc1(x))

		#Pass through Layer6
		x = self.fc2(x)

		#Return Prediction
		return x

#Create and print summary	
print("Creating EMNIST Model...")
model = EMNISTModel().to(device)
print("Model Created")

print("Model Summary: ")
summary(model,input_size=(1,28,28))

#Compile model
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Creating EMNIST Model...
Model Created
Model Summary: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
              ReLU-2           [-1, 32, 26, 26]               0
         MaxPool2d-3           [-1, 32, 13, 13]               0
            Conv2d-4           [-1, 64, 11, 11]          18,496
              ReLU-5           [-1, 64, 11, 11]               0
         MaxPool2d-6             [-1, 64, 5, 5]               0
            Conv2d-7             [-1, 64, 3, 3]          36,928
           Flatten-8                  [-1, 576]               0
            Linear-9                   [-1, 64]          36,928
             ReLU-10                   [-1, 64]               0
           Linear-11                   [-1, 26]           1,690
Total params: 94,362
Trainable params: 94,362
Non-trainable params: 0
------------------------------------------

In [14]:
#Training and saving the model

print("Training Model...")
epochs = 30

for epoch in range(epochs):
	#Train Loop

	#Set model to training mode
	model.train()
	TrainLoss = 0.0

	for i,data in enumerate(TrainLoader,0):
		inputs,labels = data[0].to(device),data[1].to(device)

		#Zero the parameter gradients
		optimizer.zero_grad()

		#Forward pass
		outputs = model(inputs)

		#Calculate loss
		loss = loss_function(outputs,labels)

		#Backward pass
		loss.backward()

		#Update weights
		optimizer.step()

		#Update loss
		TrainLoss += loss.item()
	
	#Validation Loop
	model.eval()
	ValidationLoss = 0.0
	correct = 0
	total = 0

	with torch.no_grad():
		for data in TestLoader:
			images,labels = data[0].to(device),data[1].to(device)
			outputs = model(images)
			loss = loss_function(outputs,labels)
			ValidationLoss += loss.item()

			_,predicted = torch.max(outputs.data,1)
			total += labels.size(0)
			correct += (predicted == labels).sum().item()

		acc = 100*correct/total
		print(f"Epoch: {epoch+1}/{epochs} | "
			f"Training Loss: {TrainLoss/len(TrainLoader):.3f} | "
			f"Validation Loss: {ValidationLoss / len(TestLoader):.3f} | "
			f"Accuracy: {acc:.3f}%")
		
print("Training Complete")

#Save the model
torch.save(model.state_dict(),"EMNISTModel.pth")
print("Model Saved")

Training Model...
Epoch: 1/30 | Training Loss: 0.538 | Validation Loss: 0.293 | Accuracy: 90.673%
Epoch: 2/30 | Training Loss: 0.261 | Validation Loss: 0.260 | Accuracy: 91.332%
Epoch: 3/30 | Training Loss: 0.223 | Validation Loss: 0.240 | Accuracy: 91.971%
Epoch: 4/30 | Training Loss: 0.202 | Validation Loss: 0.219 | Accuracy: 92.490%
Epoch: 5/30 | Training Loss: 0.185 | Validation Loss: 0.211 | Accuracy: 93.163%
Epoch: 6/30 | Training Loss: 0.173 | Validation Loss: 0.200 | Accuracy: 93.389%
Epoch: 7/30 | Training Loss: 0.162 | Validation Loss: 0.193 | Accuracy: 93.611%
Epoch: 8/30 | Training Loss: 0.154 | Validation Loss: 0.208 | Accuracy: 93.183%
Epoch: 9/30 | Training Loss: 0.145 | Validation Loss: 0.193 | Accuracy: 93.736%
Epoch: 10/30 | Training Loss: 0.139 | Validation Loss: 0.202 | Accuracy: 93.476%
Epoch: 11/30 | Training Loss: 0.131 | Validation Loss: 0.203 | Accuracy: 93.495%
Epoch: 12/30 | Training Loss: 0.126 | Validation Loss: 0.200 | Accuracy: 93.731%
Epoch: 13/30 | Trai

In [15]:
#Testing Model

print("Testing model with inbuilt dataset...")

model.eval()

TestLoss = 0.0
correct = 0
total = 0

with torch.no_grad():
	for images, labels in TestLoader:

		#Move data to device
		images,labels = images.to(device),labels.to(device)

		#Forward pass
		outputs = model(images)

		#Calculate loss
		loss = loss_function(outputs,labels)
		TestLoss += loss.item()

		#Get predicted class
		_,predicted = torch.max(outputs.data,1)

		#Update total and correct counts
		total += labels.size(0)
		correct += (predicted == labels).sum().item()
	
#Test Results

FinalLoss = TestLoss/len(TestLoader)
FinalAcc = 100*correct/total

print(f"Test Accuracy: {FinalAcc:.3f}%")
print(f"Test Loss: {FinalLoss:.3f}")

Testing model with inbuilt dataset...
Test Accuracy: 92.942%
Test Loss: 0.318
